# Предсказания алгоритма машинного обучения на основе данных погоды в пгт Тайга 2015-2018 г. 

In [206]:
#импорт библиотек pandas и модуля tree из библиотеки sklearn
import pandas as pd
from sklearn import tree

### Полученные данные неудобны для визуализации и формирования обучения, поэтому мы редактируем данные

In [207]:
data = pd.read_excel('2015-2018.xls', header = 6)
#data

##### Исключение ненужных столбцов
изменил столбик с осадками

In [208]:
data.drop(data.columns[[3,4,7,8,9,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]], axis=1, inplace=True)
#data.head()

##### Переименнование столбцов и выборка по дневным данным погоды

In [209]:
new_data = pd.DataFrame(columns = ['date', 'temperature', 'pressure', 'humidity', 'wind', 'cloud', 'rainfall'])

In [210]:
num = 0
row = 0
for i in data['Местное время в Тайге']:
    if '13:00' in i:
        new_data.loc[num] = [j for j in data.loc[row]]
        num += 1
    row += 1
new_data[:2]

,date,temperature,pressure,humidity,wind,cloud,rainfall
0,01.01.2019 13:00,-11.8,751.4,89.0,"Ветер, дующий с юго-запада",100%.,Состояние неба в общем не изменилось.
1,30.12.2018 13:00,-20.5,747.1,74.0,"Ветер, дующий с юго-юго-востока",Облаков нет.,


##### Перевод данных в формат для обучения методом решающих деревьев

In [211]:
## финкция преобразования русских слов в английские в столбике осадки
def translate(string):
    if ('снег' in string or 'Снег' in string) and ('дождь'in string or 'Дождь' in string):
        return 'snow with rain'
    elif 'снег' in string or 'Снег' in string:
        return 'snow'
    elif 'дождь'in string or 'Дождь' in string:
        return 'rain'
    elif 'дымка' in string or 'Дымка' in string:
        return 'smoke'
    elif 'облака'in string or 'Облака' in string:
        return 'cloud'
    elif 'туман'in string or 'Туман' in string:
        return 'fog'
    else:
        return 'clear'

In [212]:
# собираю отдельный  DataFrame где будут представленны название осадков на английском
test=[]
for k in new_data['rainfall']:
    res = translate(k)
    test.append(res)

bufer = pd.DataFrame({'rainfall':test})
bufer[:2]

,rainfall
0,clear
1,clear


In [213]:
# делаем замену столбца в исходном DataFrame
new_data.rainfall = bufer.rainfall
new_data[:10]

,date,temperature,pressure,humidity,wind,cloud,rainfall
0,01.01.2019 13:00,-11.8,751.4,89.0,"Ветер, дующий с юго-запада",100%.,clear
1,30.12.2018 13:00,-20.5,747.1,74.0,"Ветер, дующий с юго-юго-востока",Облаков нет.,clear
2,29.12.2018 13:00,-23.5,752.8,74.0,"Ветер, дующий с юго-юго-востока",Облаков нет.,smoke
3,28.12.2018 13:00,-23.7,759.4,67.0,"Штиль, безветрие",Облаков нет.,clear
4,27.12.2018 13:00,-28.2,762.2,71.0,"Ветер, дующий с востоко-северо-востока",Облаков нет.,clear
5,26.12.2018 13:00,-28.8,763.6,64.0,"Ветер, дующий с северо-востока",Облаков нет.,clear
6,25.12.2018 13:00,-27.9,762.9,68.0,"Ветер, дующий с северо-северо-востока",Облаков нет.,clear
7,24.12.2018 13:00,-31.8,759.8,67.0,"Ветер, дующий с северо-востока",Облаков нет.,clear
8,23.12.2018 13:00,-20.2,747.1,81.0,"Ветер, дующий с севера",100%.,clear
9,22.12.2018 13:00,-14.4,742.6,72.0,"Ветер, дующий с юго-запада",100%.,clear


##### Теперь мы должны сформировать матрицу обучения(X) и вектор ответов(y)
##### Объектом классификатора станет решающее дерево DecisionTreeClassifier из модуля tree. В скобках указывается глубина чтения данных для обучения

In [215]:
X = new_data.loc[:, ['temperature', 'pressure', 'humidity']]
y = new_data['rainfall']

In [216]:
classifier = tree.DecisionTreeClassifier(max_depth = 30)
#метод fit обучает алгоритм по обучающей выборке
classifier.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=30,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

##### Алгоритм обучения сформирован, теперь осталось проверить его предсказания

In [217]:
#метод predict формирует предсказание на основе матрицы обучения
prediction = classifier.predict([[-10, 730, 89]])
print(prediction)

['clear']


##### Алгоритм выдаёт нам предсказания. Но какой процент ошибки в них? Это мы можем выяснить с помощью метода k ближайших соседей(kNN)

In [218]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [219]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

##### После того, как модель обучена, мы можем предсказывать значение целевого признака по входным признакам для новых объектов. Делается это с помощью метода predict.
##### Как мы можем видеть ошибка составляет 18.4% у обучающей выборки.

In [220]:
y_predict = knn.predict(X)
err = np.mean(y != y_predict) #метод mean библиотеки numpy определяет среднее арифметическое по заданному вектору
print(err)

0.11355735805330243
